In [ ]:
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mashnoor/hls-vitis-ast-unverified-qwen-coder-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

In [ ]:
alpaca_prompt = """You are an expert in HLS C++ code development. You are provided with a design specification written originally for a Verilog module. Your task is to generate a synthesizable and functionally equivalent HLS-C++ implementation based on that specification. The generated code must adhere to high-level synthesis (HLS) best practices and include the following instruction.

    "Function Definition: Define a function or module named TopModule that maps the input and output ports as specified complying with C++ code structure. Ensure the function signature clearly represents each port.\n"
    "HLS Interface and Pragmas: Use appropriate HLS pragmas (e.g., for interface definitions and optimizations) to guide the synthesis tool. This may include directives for clock, reset, and data interfaces.\n"
    "State Management: For sequential logic (like flip-flops or registers), use static variables to preserve state between function calls, and implement edge-triggered or level-sensitive behavior as needed.\n"
    "Behavioral Equivalence: Ensure that all behavioral aspects (such as clock edge detection, asynchronous resets, and any conditional operations) from the original Verilog design are faithfully reproduced in the HLS implementation.\n"
    "Code Completeness: Provide a complete, compilable HLS-C++ code snippet that includes necessary header files and comments to clarify the mapping between the original design and the HLS implementation."

### Instruction:
{}

### Input:
{}

### Response:
{}"""
inputs = tokenizer(
[
    alpaca_prompt.format(
        # add your prompt here keeping the module name TopModule
        # one example from verilog-eval is added here
        '''
The game Lemmings involves critters with fairly simple brains. So simple
that we are going to model it using a finite state machine. In the
Lemmings' 2D world, Lemmings can be in one of two states: walking left
(walk_left is 1) or walking right (walk_right is 1). It will switch
directions if it hits an obstacle. In particular, if a Lemming is bumped
on the left (by receiving a 1 on bump_left), it will walk right. If it's
bumped on the right (by receiving a 1 on bump_right), it will walk left.
If it's bumped on both sides at the same time, it will still switch
directions.

In addition to walking left and right and changing direction when bumped,
when ground=0, the Lemming will fall and say "aaah!". When the ground
reappears (ground=1), the Lemming will resume walking in the same
direction as before the fall. Being bumped while falling does not affect
the walking direction, and being bumped in the same cycle as ground
disappears (but not yet falling), or when the ground reappears while
still falling, also does not affect the walking direction.

In addition to walking and falling, Lemmings can sometimes be told to do
useful things, like dig (it starts digging when dig=1). A Lemming can dig
if it is currently walking on ground (ground=1 and not falling), and will
continue digging until it reaches the other side (ground=0). At that
point, since there is no ground, it will fall (aaah!), then continue
walking in its original direction once it hits ground again. As with
falling, being bumped while digging has no effect, and being told to dig
when falling or when there is no ground is ignored. (In other words, a
walking Lemming can fall, dig, or switch directions. If more than one of
these conditions are satisfied, fall has higher precedence than dig,
which has higher precedence than switching directions.)

Implement a Moore state machine that models this behaviour. areset is
positive edge triggered asynchronous reseting the Lemming machine to walk
left.

module TopModule (
  input clk,
  input areset,
  input bump_left,
  input bump_right,
  input ground,
  input dig,
  output walk_left,
  output walk_right,
  output aaah,
  output digging
);
        ''', # instruction
        '''

        ''', # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 30000, temperature=1.0, do_sample=True)

In [ ]:
alpaca_prompt = """Your role is to act as an expert in HLS C++ code development. You must thoroughly explore each question through a systematic
and deliberate thinking process—engaging in cycles of analysis, summarization, exploration, reassessment, reflection, backtracing, and
iteration—to develop well-considered solutions. Based on the provided instructions, you are expected to generate precise, optimized, and accurate
HLS C++ code that ideally meets the requirements. You must define a function named TopModule that
maps the input and output ports ensuring the function signature that clearly represents each port in the instruction.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
inputs = tokenizer(
[
    alpaca_prompt.format(
        '''
Consider the FSM described by the state diagram shown below:

  A --r1=0,r2=0,r3=0--> A
  A --r1=1--> B
  A --r1=0,r2=1--> C
  A --r1=0,r2=0,r3=0--> D
  B (g1=1) --r1=1--> B
  B (g1=1) --r1=0--> A
  C (g2=1) --r2=1--> C
  C (g2=1) --r2=0--> A

Resetn is an active-low synchronous reset that resets into state A. This
FSM acts as an arbiter circuit, which controls access to some type of
resource by three requesting devices. Each device makes its request for
the resource by setting a signal _r[i]_ = 1, where _r[i]_ is either
_r[1]_, _r[2]_, or _r[3]_. Each r[i] is an input signal to the FSM, and
represents one of the three devices. The FSM stays in state _A_ as long
as there are no requests. When one or more request occurs, then the FSM
decides which device receives a grant to use the resource and changes to
a state that sets that device's _g[i]_ signal to 1. Each _g[i]_ is an
output from the FSM. There is a priority system, in that device 1 has a
higher priority than device 2, and device 3 has the lowest priority.
Hence, for example, device 3 will only receive a grant if it is the only
device making a request when the FSM is in state _A_. Once a device, _i_,
is given a grant by the FSM, that device continues to receive the grant
as long as its request, _r[i]_ = 1.

Write complete Verilog code that represents this FSM. Use separate always
blocks for the state table and the state flip-flops, as done in lectures.
Describe the FSM outputs, _g[i]_, using either continuous assignment
statement(s) or an always block (at your discretion). Assign any state
codes that you wish to use.

module TopModule (
  input clk,
  input resetn,
  input [3:1] r,
  output [3:1] g
);
    ''', # instruction
        '''

        ''', # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 30000, temperature=1.0, do_sample=True)